In [1]:
import os
import librosa
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Path to your audio files
DATA_PATH = r"C:\Users\vaidi\Downloads\emotion_detection\AudioWAV"

# Allowed emotion labels
TARGET_EMOTIONS = {'ANG', 'DIS', 'FEA', 'HAP', 'SAD'}

X = []
y = []

# Load each WAV file and extract features
for file in os.listdir(DATA_PATH):
    if file.endswith(".wav"):
        emotion = file.split('_')[2]
        if emotion not in TARGET_EMOTIONS:
            continue

        file_path = os.path.join(DATA_PATH, file)
        try:
            audio, sr = librosa.load(file_path, sr=None)

            # Extract 40 MFCCs
            mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
            delta = librosa.feature.delta(mfcc)
            delta2 = librosa.feature.delta(mfcc, order=2)

            # Concatenate features (MFCC + delta + delta-delta)
            combined = np.concatenate((mfcc, delta, delta2), axis=0)
            mean_features = np.mean(combined.T, axis=0)

            X.append(mean_features)
            y.append(emotion)
        except Exception as e:
            print(f"Error processing {file}: {e}")

X = np.array(X)
y = np.array(y)

print(f"Total samples: {len(X)}")
print(f"Labels: {set(y)}")

Total samples: 6355
Labels: {np.str_('FEA'), np.str_('ANG'), np.str_('DIS'), np.str_('SAD'), np.str_('HAP')}


In [2]:
# Encode string labels to integers
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encode the integer labels
y_onehot = to_categorical(y_encoded)

# Train-test split (stratify maintains label proportions)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_onehot, test_size=0.2, random_state=42, stratify=y_encoded
)

print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])
print("Classes:", label_encoder.classes_)


Training samples: 5084
Testing samples: 1271
Classes: ['ANG' 'DIS' 'FEA' 'HAP' 'SAD']


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Build the model
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(y_train.shape[1], activation='softmax')  # Output layer for 5 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_test, y_test)
)


C:\Users\vaidi\anaconda3\envs\emoenv\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.2078 - loss: 13.1124 - val_accuracy: 0.1998 - val_loss: 1.6086
Epoch 2/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2385 - loss: 1.6620 - val_accuracy: 0.2360 - val_loss: 1.5966
Epoch 3/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2440 - loss: 1.6038 - val_accuracy: 0.2801 - val_loss: 1.5603
Epoch 4/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.2783 - loss: 1.5569 - val_accuracy: 0.3714 - val_loss: 1.4667
Epoch 5/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3073 - loss: 1.5359 - val_accuracy: 0.3682 - val_loss: 1.4425
Epoch 6/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3104 - loss: 1.4980 - val_accuracy: 0.3729 - val_loss: 1.4361
Epoch 7/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3388 - loss: 1.4668 - val_accuracy: 0.3745 - val_loss: 1.4128
Epoch 8/100
159/159 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3358 - loss: 1.4458 - val_acc

In [4]:
# Evaluate on test data
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4850 - loss: 1.2591
Test Accuracy: 49.49%


In [5]:
# Save the model to a file
model.save("emotion_detection_upgrademodel.h5")
print("Model saved successfully!")


Model saved successfully!
